# libs

In [1]:
from scipy.stats import nbinom, binom, betabinom
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
from plotly.subplots import make_subplots
import pandas as pd
from collections import Counter
import scipy.integrate as it
import numpy as np
import time 



# Variaveis

In [2]:
n = 100
observed = 50
Experimento = {
    'Grupo 1': {
        'Modelo 1': {'K': 0, 'a': 400, 'b': 400},
        'Modelo 2': {'K': 0, 'a': 50, 'b': 50},
        'Modelo 3': {'K': 0, 'a': 20, 'b': 20}
    },
    'Grupo 2': {
        'Modelo 1': {'K': 0, 'a': 8, 'b': 11},
        'Modelo 2': {'K': 0, 'a': 11, 'b': 8}
    },
    'Grupo 3': {
        'Modelo 1': {'K': 0, 'a': 25, 'b': 25},
        'Modelo 2': {'K': 5, 'a': 25, 'b': 25},
        'Modelo 3': {'K': 10, 'a': 25, 'b': 25},
        'Modelo 4': {'K': 15, 'a': 25, 'b': 25}
    },
    'Grupo 4': {
        'Modelo 1': {'K': 0,  'a': 25, 'b': 25},
        'Modelo 2': {'K': 40, 'a': 25, 'b': 25},
        'Modelo 3': {'K': 60, 'a': 25, 'b': 25}
    },
}


In [3]:
def pinball_loss(observed, estimate, alpha):
    if observed >= estimate: 
        result = (observed-estimate)*alpha
    else:
        result = (estimate-observed)*(1-alpha)
    return result

alphas = np.linspace(0, 1, 1_000)

weighting_function = {
    "two-tailed": (alphas-.5)**2,
    "left-tail": (1-alphas)**2,
    "right-tail": (alphas)**2,
    "sem peso": np.ones(len(alphas)),
}

# Gerando distribuições

In [4]:
resultados = []
df_densidade = pd.DataFrame()
for nome, grupo in Experimento.items():
    max_x = n + max([param['K'] for param in grupo.values()])
    for modelo, param in grupo.items():
        resultado = {'grupo': nome, 'modelo': modelo, **param}
        resultado['n'] = n
        r = param.copy()
        Z = betabinom(n=n, a=param['a'], b=param['b'])
        # salvando distribuição
        x = list(range(0, max_x))
        y = Z.pmf(x)
        x = [x_i + param['K'] for x_i in x]
        p_z = {x_i: y_i for x_i, y_i in zip(x, y)}
        df_modelo = pd.DataFrame({'x': x, 'y': y}).assign(modelo=modelo, grupo=nome)
        df_modelo = df_modelo.loc[lambda df: df['y'] > 0]
        df_densidade = pd.concat([df_densidade,df_modelo])
        # calculando densidade
        L =  - 2*p_z.get(observed, 0)
        for z, p in p_z.items():
            L += (p**2)/len(p_z)
        resultado['Densidade'] = L
        # calculando perdas pinball
        L_alpha = [
            pinball_loss(
                observed = observed, 
                estimate = (Z.ppf(a) + param.get('K', 0)), 
                alpha=a
                ) 
            for a in alphas]
        for name, weight in weighting_function.items():
            resultado[name] = it.simps(weight*L_alpha, alphas)
        # calculando MSE
        resultado['MSE'] = (observed-Z.mean()+param.get('K', 0))**2
        # salvando resultados
        resultados.append(resultado)


In [15]:
fig = make_subplots(
    rows=2, cols=2, shared_xaxes=False, shared_yaxes=False,
    subplot_titles=['(a)', '(b)', '(c)', '(d)'])

plots = {
    'Grupo 1': {'row': 1, 'col': 1},
    'Grupo 2': {'row': 1, 'col': 2},
    'Grupo 3': {'row': 2, 'col': 1},
    'Grupo 4': {'row': 2, 'col': 2},
}
colors = ['#1F78B4', '#FE7F0E', '#2BA02D', '#C03D3E']

for grupo, axis in plots.items():
    df_grupo = df_densidade.loc[lambda df: df['grupo'] == grupo]
    max_y = df_grupo['y'].max()
    max_x = n + max([param['K'] for param in Experimento[grupo].values()])
    for i, modelo in enumerate(df_grupo['modelo'].unique()):
        df_modelo = df_grupo.loc[lambda df: df['modelo'] == modelo]
        fig_ = go.Scatter(
            x=df_modelo['x'], y=df_modelo['y']*100,
            mode='lines+markers', 
            line=dict(color=colors[i], width=1),
            marker=dict(color=colors[i], size=4),
            name=modelo,
            )
        if grupo != 'Grupo 3':
            fig_.update(showlegend=False)
        fig.add_trace(fig_,  **axis)
    fig_ = go.Scatter(
        x=[observed, observed], 
        y=[0,max_y*100], 
        mode='lines', 
        line=dict(color='black', width=2, dash='dash'),
        name='Observado'
        )

    if grupo != 'Grupo 3':
        fig_.update(showlegend=False)
    fig.add_trace(fig_, **axis)

    fig.update_xaxes(range=[0, max_x], **axis)


fig.update_layout(
    template = "simple_white",
    #width=800, height=800
    )

fig.update_xaxes(title_text='Tempo de execução do projeto (dias)', row=2, col=1)
fig.update_xaxes(title_text='Tempo de execução do projeto (dias)', row=2, col=2)
fig.update_yaxes(title_text='Probabilidade (%)', row=1, col=1)
fig.update_yaxes(title_text='Probabilidade (%)', row=2, col=1)

plot_url = py.plot(fig, filename='latex', include_mathjax='cdn')

In [6]:
print(f"Grupo & Modelo & K & n & a & b & MSE & Densidade & Sem peso & Duas caudas & Cauda esquerda & Cauda direita \\\\")
for resultado in resultados:
    print(f"{resultado['grupo']} & {resultado['modelo']} & {resultado['K']} & {n} & {resultado['a']} & {resultado['b']} & {resultado['MSE']:.0f} & {resultado['Densidade']:.3f} & {resultado['sem peso']:.2f} & {resultado['two-tailed']:.2f} & {resultado['left-tail']:.2f} & {resultado['right-tail']:.2f} \\\\")

Grupo & Modelo & K & n & a & b & MSE & Densidade & Sem peso & Duas caudas & Cauda esquerda & Cauda direita \\
Grupo 1 & Modelo 1 & 0 & 100 & 400 & 400 & 0 & -0.150 & 0.62 & 0.05 & 0.20 & 0.20 \\
Grupo 1 & Modelo 2 & 0 & 100 & 50 & 50 & 0 & -0.112 & 0.82 & 0.07 & 0.27 & 0.27 \\
Grupo 1 & Modelo 3 & 0 & 100 & 20 & 20 & 0 & -0.084 & 1.09 & 0.09 & 0.36 & 0.36 \\
Grupo 2 & Modelo 1 & 0 & 100 & 8 & 11 & 62 & -0.051 & 2.47 & 0.13 & 0.99 & 0.50 \\
Grupo 2 & Modelo 2 & 0 & 100 & 11 & 8 & 62 & -0.051 & 2.47 & 0.13 & 0.50 & 0.99 \\
Grupo 3 & Modelo 1 & 0 & 100 & 25 & 25 & 0 & -0.091 & 1.01 & 0.08 & 0.34 & 0.34 \\
Grupo 3 & Modelo 2 & 5 & 100 & 25 & 25 & 25 & -0.078 & 1.57 & 0.09 & 0.31 & 0.65 \\
Grupo 3 & Modelo 3 & 10 & 100 & 25 & 25 & 100 & -0.048 & 3.09 & 0.14 & 0.77 & 1.05 \\
Grupo 3 & Modelo 4 & 15 & 100 & 25 & 25 & 225 & -0.021 & 5.21 & 0.26 & 1.67 & 1.46 \\
Grupo 4 & Modelo 1 & 0 & 100 & 25 & 25 & 0 & -0.091 & 1.01 & 0.08 & 0.34 & 0.34 \\
Grupo 4 & Modelo 2 & 40 & 100 & 25 & 25 & 1600 & 0.